In [29]:
### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict
import sys
sys.path.append('/mnt/raid1/Python_projects')
sys.path.append('/mnt/raid1/Python_projects/pkg_mod')

# DATABASE Connections

# Chemistry Database
#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print(datetime.now())

###################################################################################################################

### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###

import collections
import fingerprinter as fpt

# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_fingerprint fp WHERE f.structure_id = fp.structure_id))' 
cur.execute(sql) 

mollist = cur.fetchall()
print()
print('##### Structures without Fingerprints #####')
print()
print(len(mollist),'new Structures')
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
                
print()        
# print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
###################################################################################################################

### Step 3: Insert new fingerprints into fingerprint table ###

print()
print('##### Insert New Fingerprints #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public.fingerprints (fp_smiles, fp_number, fp_type) VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        print(fpb,' - ',v, smi_fl, "inserted")
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        #print(fpb,' - ',v, smi_fl, "inserted")

###################################################################################################################

### Step 4: Insert Fingerprints for New Molecules ###

print()
print('##### Insert Structure/Fingerprints Links #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public.structure_fingerprint (structure_id,fp_id,fp_count) VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    
db_chem.close()
    
###################################################################################################################

### Step 5: Update Shannon Entropies ###
print()
print('##### Shannon Update #####')
print()

# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
    print()
else:
    #print('New structure/fingerprint links')
    #db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
    db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
    cur = db_chem.cursor() 
    sql = 'SELECT sf.structure_id, sf.fp_id, sf.fp_count, fp.fp_shannon FROM public.structure_fingerprint sf \
    JOIN public.fingerprints fp ON sf.fp_id = fp.fingerprint_id WHERE fp.fp_type = 1'
    cur.execute(sql)
    fp_list = cur.fetchall()
    # print(fp_list)

    fp_dict = collections.defaultdict(list)
    fp_mol = collections.defaultdict(list)

    for m in fp_list:
        # print(m, m[1],m[2])
        fp_dict[m[1]].append(m[2])
        fp_mol[m[0]].append(m[1])

    # n_fp = 0
    print(len(fp_dict),'fingerprints')
    print(len(fp_mol),'structures')
    print()

    n_fp = len(fp_mol)

    #for fpb, i in fp_dict.items():
    #    n_fp = max(n_fp, len(i))
        # print (fpb, i, n)


    # update entropy field in Fingerprints    
    
    sql = 'UPDATE public.fingerprints SET fp_shannon = %s WHERE fingerprint_id = %s'
    cur = db_chem.cursor()
    for fpb, i in fp_dict.items():
        ns = netprepper.shannon(i,n_fp)
        # print(fpb,' - ',netprepper.shannon(i,n))
        print(fpb,' - ',ns[0],' - ',ns[1])
        cur.execute(sql, (ns[1], fpb))

    db_chem.commit()
    db_chem.close()

###################################################################################################################

### Step 6: Update Tanimoto ###
print()
print('##### Tanimoto Update #####')
print()

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 

#sql = 'select * FROM public.tanimoto'
#cur.execute(sql)
#q = cur.fetchall()
#print(q)
#print()

sql = 'SELECT max(target) FROM public.tanimoto'
cur.execute(sql)
t = cur.fetchone()
#print(tani_max, type(tani_max))
if t is None:
    tani_max = 0
else:
    tani_max = t[0]
#print(tani_max)

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'SELECT max(structure_id) FROM public.structures' 
cur.execute(sql) 
m = cur.fetchone()
mol_max = m[0]
print('TaniMax : ',tani_max,' - Mol_max : ',mol_max)

if mol_max > tani_max:
    ml = len(mollist)
    tp = mollist[0][0]
    i = 0
    while tp < tani_max:
        tp = mollist[i][0]
        i = i+1
    print(ml,tp, i)
    tp = i
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    for i in range(tp+1, ml):
        m2 = Chem.MolFromSmiles(mollist[i][1])
        fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
        for j in range (0, i):
            m1 = Chem.MolFromSmiles(mollist[j][1])
            fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
            tan = DataStructs.TanimotoSimilarity(fp1,fp2)
            if tan >= .5:
                #print(i, j, tan, mollist[i][0], mollist[i][1], mollist[j][0], mollist[j][1])
                print(mollist[j][0],mollist[i][0], tan)
                cur.execute(sql, (mollist[j][0],mollist[i][0], tan))

db_chem.commit()
db_chem.close()


PostgreSQL database version: ('PostgreSQL 12.8 (Ubuntu 12.8-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
2021-09-24 22:43:44.767266

##### Structures without Fingerprints #####

0 new Structures
[]

All Fingerprints by molecule
[]

{}
0 SMILES

0 fingerprints

##### Insert New Fingerprints #####


No new fingerprint inserts


##### Insert Structure/Fingerprints Links #####


No new structure/fingerprint inserts


##### Shannon Update #####

No new structure/fingerprint inserts


##### Tanimoto Update #####

TaniMax :  118  - Mol_max :  118


In [23]:
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 

sql = 'select * FROM public.tanimoto'
cur.execute(sql)
q = cur.fetchall()
print(q)
print()

sql = 'SELECT max(target) FROM public.tanimoto'
cur.execute(sql)
t = cur.fetchone()
#print(tani_max, type(tani_max))
if t is None:
    tani_max = 0
else:
    tani_max = t[0]
#print(tani_max)

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'SELECT max(structure_id) FROM public.structures' 
cur.execute(sql) 
m = cur.fetchone()
mol_max = m[0]
print(tani_max, mol_max)

if mol_max > tani_max:
    ml = len(mollist)
    tp = mollist[0][0]
    i = 0
    while tp < tani_max:
        tp = mollist[i][0]
        i = i+1
    print(ml,tp, i)
    tp = i
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    for i in range(tp+1, ml):
        m2 = Chem.MolFromSmiles(mollist[i][1])
        fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
        for j in range (0, i):
            m1 = Chem.MolFromSmiles(mollist[j][1])
            fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
            tan = DataStructs.TanimotoSimilarity(fp1,fp2)
            if tan >= .5:
                #print(i, j, tan, mollist[i][0], mollist[i][1], mollist[j][0], mollist[j][1])
                print(mollist[j][0],mollist[i][0], tan)
                cur.execute(sql, (mollist[j][0],mollist[i][0], tan))

db_chem.commit()
db_chem.close()
            #print(fp1, fp2)
            
#fp1 = Chem.RDKFingerprint(ref)
#fp2 = Chem.RDKFingerprint(mol1)

[(13, 16, 0.5), (34, 38, 0.5263157894736842), (42, 43, 0.8852459016393442), (42, 44, 0.8360655737704918), (43, 44, 0.7692307692307693), (42, 45, 0.75), (43, 45, 0.7164179104477612), (44, 45, 0.75), (42, 46, 0.703125), (43, 46, 0.6716417910447762), (44, 46, 0.703125), (45, 46, 0.8166666666666667), (47, 48, 0.5619047619047619), (56, 57, 0.7966101694915254), (58, 59, 0.88), (58, 60, 0.8076923076923077), (59, 60, 0.88), (58, 61, 0.88), (59, 61, 1.0), (60, 61, 0.88), (58, 62, 0.88), (59, 62, 1.0), (60, 62, 0.88), (61, 62, 1.0), (79, 84, 0.5652173913043478), (79, 85, 0.5319148936170213), (79, 86, 0.5319148936170213), (84, 86, 0.627906976744186), (66, 102, 0.6), (101, 102, 0.5384615384615384), (101, 103, 0.5384615384615384), (102, 103, 0.5384615384615384), (108, 109, 0.5897435897435898), (67, 110, 0.5), (67, 112, 0.5), (110, 112, 0.6), (112, 113, 0.6), (110, 114, 0.574468085106383), (112, 114, 0.574468085106383), (65, 115, 0.5), (113, 115, 0.6), (65, 116, 0.5), (115, 116, 0.6), (108, 118, 0.6